In [1]:
import os
os.chdir('/home/s3/hyeryung/mucoco')

import pandas as pd

# Debug Issue 1. Low Bertscore (negative) for CLSF_BV0 & EM_BV0
## Merge all results into one file.

In [17]:
# with open('data/Sentiment-and-Style-Transfer/data/yelp/sentiment.test.0') as f:
#     source_0 = [line.rstrip('\n') for line in f.readlines()]

In [78]:
ref_0 = pd.read_csv('data/Sentiment-and-Style-Transfer/data/yelp/reference.0',sep='\t',header=None,names=['text','ref'])

In [80]:
with open('lewis/model_outputs/yelp/pos.out', 'r') as f:
    lewis_0 = [line.rstrip('\n') for line in f.readlines()]
ref_0['lewis']=lewis_0

In [81]:
clsf_bv0 = pd.read_json('outputs/sentiment/mlm-reranking/roberta-base-yelp-lewis-sentiment-classifier-with-gpt2-large-embeds-binary/lewis-compr/mlm-beamsearch-v0-word-nps5-k10-beam5-allsat_primary-vtwtoeyi-negative-to-positive/outputs_epsilon0.75.txt',
                        lines=True)

## unravel data
clsf_bv0=clsf_bv0.explode('generations')
clsf_bv0['text']=clsf_bv0['generations'].apply(lambda x: x['text'])

ref_0['clsf_bv0']=clsf_bv0['text']

In [82]:
em_bv0 = pd.read_json('outputs/sentiment/mlm-reranking/roberta-base-yelp-sentiment-classifier-with-gpt2-large-embeds-energy-training/lewis-compr/mlm-beamsearch-v0-word-nps5-k10-beam5-allsat_primary-dppnf7pu-negative-to-positive/outputs_epsilon0.75.txt', 
                      lines=True)

## unravel data
em_bv0=em_bv0.explode('generations')
em_bv0['text']=em_bv0['generations'].apply(lambda x: x['text'])

ref_0['em_bv0']=em_bv0['text']

In [83]:
lewis_bert = pd.read_csv('lewis/model_outputs/yelp/results-negative-to-positive.txt.bertscore')
ref_0['bert_lewis']=lewis_bert['bert_score']
ref_0['sbert_lewis']=lewis_bert['sbert_score']

In [84]:
lewis_cls = pd.read_csv('lewis/model_outputs/yelp/results-negative-to-positive.txt.cls')
ref_0['cls_lewis']=lewis_cls['sentiment_predictions']

In [85]:
clsf_bv0_bert = pd.read_csv('outputs/sentiment/mlm-reranking/roberta-base-yelp-lewis-sentiment-classifier-with-gpt2-large-embeds-binary/lewis-compr/mlm-beamsearch-v0-word-nps5-k10-beam5-allsat_primary-epsilon0.75-results-negative-to-positive.txt.bertscore')
ref_0['bert_clsf_bv0']=clsf_bv0_bert['bert_score']
ref_0['sbert_clsf_bv0']=clsf_bv0_bert['sbert_score']

In [86]:
clsf_bv0_cls = pd.read_csv('outputs/sentiment/mlm-reranking/roberta-base-yelp-lewis-sentiment-classifier-with-gpt2-large-embeds-binary/lewis-compr/mlm-beamsearch-v0-word-nps5-k10-beam5-allsat_primary-epsilon0.75-results-negative-to-positive.txt.cls')
ref_0['cls_clsf_bv0']=clsf_bv0_cls['sentiment_predictions']

In [87]:
em_bv0_bert = pd.read_csv('outputs/sentiment/mlm-reranking/roberta-base-yelp-sentiment-classifier-with-gpt2-large-embeds-energy-training/lewis-compr/mlm-beamsearch-v0-word-nps5-k10-beam5-allsat_primary-epsilon0.75-results-negative-to-positive.txt.bertscore')
ref_0['bert_em_bv0']=em_bv0_bert['bert_score']
ref_0['sbert_em_bv0']=em_bv0_bert['sbert_score']

In [88]:
em_bv0_cls = pd.read_csv('outputs/sentiment/mlm-reranking/roberta-base-yelp-sentiment-classifier-with-gpt2-large-embeds-energy-training/lewis-compr/mlm-beamsearch-v0-word-nps5-k10-beam5-allsat_primary-epsilon0.75-results-negative-to-positive.txt.cls')
ref_0['cls_em_bv0']=em_bv0_cls['sentiment_predictions']

In [94]:
ref_0.to_excel('new_module/compr/evaluate_lewis.xlsx')

## Found Issue 1. For EM_BV0, it seems that quite a lot of samples are '' -> which result in -4.925 in bert scores. 
Solution: rerun decoding so that regardless of initial text's satisfaction of constraint, locate&edit is conducted for all samples.

## Found Issue 2. For CLSF_BV0, strangely, the rows with -4.925 BertScore has string. Find out why. -> Found out that I reversed the pred_0 and pred_1 when evaluating clsf_bv0. 
Solution: rerun decoding with correct files.

In [12]:
import evaluate

In [5]:
ref_0=pd.read_excel('new_module/compr/evaluate_lewis.xlsx',index_col=0)

In [7]:
debug_ref_0 = ref_0.loc[ref_0['bert_clsf_bv0']<-3,:].copy()

In [8]:
debug_ref_0

,text,ref,lewis,clsf_bv0,em_bv0,bert_lewis,sbert_lewis,cls_lewis,bert_clsf_bv0,sbert_clsf_bv0,cls_clsf_bv0,bert_em_bv0,sbert_em_bv0,cls_em_bv0
13,said we could n't sit at the table if we were ...,they said we could sit at the table with no he...,we were so impressed we could n't sit at the t...,"""We couldn't be at the table if we weren't in ...",I think we have to sit at the table if we want...,0.318293,0.635969,1,-4.925075,-4.925075,1,0.336163,0.319579,0
18,the burgers were over cooked to the point the ...,the burgers were cooked perfectly and the meat...,the burgers were perfectly cooked to the point...,But they were so close to the point the game w...,the burgers have been cooked to the point the ...,0.748451,0.790539,1,-4.925075,-4.925075,1,0.588865,0.810976,0
56,lost a long time customer !,gained a long time customer !,i will be a long time customer !,"Hi, first time customer!\n",NaN,0.630864,0.624011,1,-4.925075,-4.925075,1,-4.925075,-4.925075,1
59,so far i 'm not really impressed .,so far i 'm really impressed .,so far i am impressed .,So far i'm not really sure.\n,"Well, it's really really good.\n",0.884421,0.759502,1,-4.925075,-4.925075,1,0.392473,0.331879,1
83,"the salads are ok , nothing special .","the salads are great, really something special.",the salads are always special .,"If you are ok, nothing special.\n","the kids are good, I think.\n",0.618087,0.566846,1,-4.925075,-4.925075,1,0.416270,0.386847,1
88,the garlic bread was bland and cold .,it was sitting out for a while on the counter ...,the garlic bread was fresh and delicious .,My garlic bread was bland and cold.\n,"For the bread, take the bread.\n",0.070940,0.752917,1,-4.925075,-4.925075,1,-0.037768,0.141551,1
92,i love the food ... however service here is ho...,i love the food ... and service here is aweso...,i love the food and the service here is great .,I love the food... however service here is hor...,i love the place... however service here is ho...,0.851289,0.690451,1,-4.925075,-4.925075,1,0.644619,0.889690,0
97,she did not apologize or anything .,she apologized immensily,she did a great job or anything .,He did not die or anything.\n,she did not apologize or resign.\n,0.085278,0.432117,1,-4.925075,-4.925075,1,0.332343,0.649234,1
104,we 've tried the cream pancakes as well .,We loved the cream pancakes too.,we loved the cream pancakes as well .,I've tried the cream pancakes as well.\n,NaN,0.959326,0.777983,1,-4.925075,-4.925075,1,-4.925075,-4.925075,1
129,just went back to get the rear tire sealed and...,went back and they helped me quickly on the fi...,just came back to get the rear tire sealed and...,I went back to check the rear view mirror and ...,NaN,0.242730,0.847873,0,-4.925075,-4.925075,1,-4.925075,-4.925075,1


In [14]:
idx = 13
src = debug_ref_0.loc[idx, 'text']
ref = debug_ref_0.loc[idx, 'ref']
pred = debug_ref_0.loc[idx, 'clsf_bv0']

In [13]:
bertscore = evaluate.load("bertscore")

In [16]:
os.environ['HF_HOME']='/shared/s3/lab07/hyeryung/hf_cache'
os.environ['HF_DATASETS_CACHE']='/shared/s3/lab07/hyeryung/hf_cache'
os.environ['TRANSFORMERS_CACHE']='/shared/s3/lab07/hyeryung/hf_cache'

In [17]:
bert_score = bertscore.compute(predictions=[pred], references=[ref], lang="en", rescale_with_baseline=True)

model.safetensors:   0%|          | 0.00/1.42G [00:00<?, ?B/s]

Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['roberta.pooler.dense.weight', 'roberta.pooler.dense.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [18]:
bert_score

{'precision': [0.1198190376162529],
 'recall': [0.22995620965957642],
 'f1': [0.17563839256763458],
 'hashcode': 'roberta-large_L17_no-idf_version=0.3.12(hug_trans=4.30.2)-rescaled'}

In [19]:
bert_score = bertscore.compute(predictions=[pred], references=[src], lang="en", rescale_with_baseline=True)

In [20]:
bert_score

{'precision': [0.3526301085948944],
 'recall': [0.18894222378730774],
 'f1': [0.27068275213241577],
 'hashcode': 'roberta-large_L17_no-idf_version=0.3.12(hug_trans=4.30.2)-rescaled'}

In [24]:
bert_score = bertscore.compute(predictions=debug_ref_0.loc[:, 'clsf_bv0'].tolist(), references=debug_ref_0.loc[:, 'text'].tolist(), lang="en", rescale_with_baseline=True)

In [25]:
bert_score

{'precision': [0.35263150930404663,
  0.19314943253993988,
  0.33724766969680786,
  0.6753238439559937,
  0.5138356685638428,
  0.9661069512367249,
  0.8884523510932922,
  0.6810457110404968,
  0.9327162504196167,
  0.39812788367271423,
  0.6221935749053955,
  0.26614412665367126,
  0.46920350193977356,
  0.7260136008262634,
  0.5426690578460693,
  0.19026567041873932,
  0.8892121911048889,
  0.8255161046981812,
  0.7217661142349243,
  0.44464200735092163,
  0.6815459132194519,
  0.6650265455245972,
  0.684224545955658,
  0.4952433109283447,
  0.7666742205619812,
  0.6958027482032776],
 'recall': [0.1889432817697525,
  0.11681004613637924,
  0.35667726397514343,
  0.44781434535980225,
  0.3449268937110901,
  0.8324404358863831,
  0.8002338409423828,
  0.5398414134979248,
  0.6453701257705688,
  0.2736797332763672,
  0.5911695957183838,
  0.09406804293394089,
  0.24916626513004303,
  0.6034619212150574,
  0.4189896881580353,
  0.05509646236896515,
  0.7461383938789368,
  0.6725088953971

# Debug Issue 2. Low Accuracy Score for CLSF_BV0

In [4]:
clsf_bv0 = pd.read_json('outputs/sentiment/mlm-reranking/roberta-base-yelp-lewis-sentiment-classifier-with-gpt2-large-embeds-binary/lewis-compr/mlm-beamsearch-v0-word-nps5-k10-beam5-allsat_primary-9fjc2pfg-negative-to-positive/outputs_epsilon0.75.txt',lines=True)

In [6]:
clsf_bv0=clsf_bv0.explode('generations')
clsf_bv0['text']=clsf_bv0['generations'].apply(lambda x: x['text'])

In [9]:
clsf_bv0['indices']=clsf_bv0['generations'].apply(lambda x: x['indices'])
clsf_bv0['orig_tokens_at_indices']=clsf_bv0['generations'].apply(lambda x: x['orig_tokens_at_indices'])

In [13]:
ref_0 = pd.read_csv('data/Sentiment-and-Style-Transfer/data/yelp/reference.0',sep='\t',header=None,names=['src','ref'])

In [20]:
clsf_bv0_cls = pd.read_csv('outputs/sentiment/mlm-reranking/roberta-base-yelp-lewis-sentiment-classifier-with-gpt2-large-embeds-binary/lewis-compr/mlm-beamsearch-v0-word-nps5-k10-beam5-allsat_primary-epsilon0.75-results-negative-to-positive.txt.cls')

In [16]:
debug_df=pd.concat([ref_0,clsf_bv0[['text','indices','orig_tokens_at_indices']]],axis=1)

In [21]:
debug_df['cls_clsf_bv0']=clsf_bv0_cls['sentiment_predictions']

In [23]:
debug_df.to_excel('new_module/compr/result_debug_clsf_bv0.xlsx',index=False)

# Code to Generate Excel File for Qualitative Result Anlaysis

In [17]:
# with open('data/Sentiment-and-Style-Transfer/data/yelp/sentiment.test.0') as f:
#     source_0 = [line.rstrip('\n') for line in f.readlines()]

In [262]:
ref_0 = pd.read_csv('data/Sentiment-and-Style-Transfer/data/yelp/reference.0',sep='\t',header=None,names=['src','ref'])

In [263]:
cls=pd.read_csv('data/Sentiment-and-Style-Transfer/data/yelp/reference.0.results.txt.cls')
ref_0[f'ref_cls']=cls['sentiment_predictions']

In [264]:
with open('lewis/model_outputs/yelp/pos.out', 'r') as f:
    lewis_0 = pd.DataFrame({"lewis":[line.rstrip('\n') for line in f.readlines()]})

In [265]:
cls=pd.read_csv('lewis/model_outputs/yelp/results-negative-to-positive.txt.cls')
lewis_0[f'lewis_cls']=cls['sentiment_predictions']

In [277]:
result_paths_0 = ['outputs/sentiment/mlm-reranking/roberta-base-yelp-lewis-sentiment-classifier-with-gpt2-large-embeds-binary/lewis-compr/mlm-beamsearch-v0-word-nps5-k10-beam5-allsat_primary-866hmm6j-negative-to-positive-grad_norm-866hmm6j/outputs_epsilon0.75.txt',
                'outputs/sentiment/mlm-reranking/roberta-base-yelp-lewis-sentiment-classifier-with-gpt2-large-embeds-binary/lewis-compr/mlm-beamsearch-v0-word-nps5-k10-beam5-allsat_primary-9fjc2pfg-negative-to-positive/outputs_epsilon0.75.txt',
               # 'outputs/sentiment/mlm-reranking/roberta-base-yelp-sentiment-classifier-with-gpt2-large-embeds-energy-training/lewis-compr/mlm-beamsearch-v0-word-nps5-k10-beam5-allsat_primary-k5ojm8zg-negative-to-positive-grad_norm-k5ojm8zg/outputs_epsilon0.75.txt',
               # 'outputs/sentiment/mlm-reranking/roberta-base-yelp-sentiment-classifier-with-gpt2-large-embeds-energy-training/lewis-compr/mlm-beamsearch-v0-word-nps5-k10-beam5-allsat_primary-z9nof4p7-negative-to-positive/outputs_epsilon0.75.txt'
                 ]

In [278]:
cls_paths_0 = ['outputs/sentiment/mlm-reranking/roberta-base-yelp-lewis-sentiment-classifier-with-gpt2-large-embeds-binary/lewis-compr/mlm-beamsearch-v0-word-nps5-k10-beam5-allsat_primary-866hmm6j-negative-to-positive-grad_norm-866hmm6j/results_epsilon0.75.txt.cls',
                'outputs/sentiment/mlm-reranking/roberta-base-yelp-lewis-sentiment-classifier-with-gpt2-large-embeds-binary/lewis-compr/mlm-beamsearch-v0-word-nps5-k10-beam5-allsat_primary-9fjc2pfg-negative-to-positive/results_epsilon0.75.txt.cls',
               # 'outputs/sentiment/mlm-reranking/roberta-base-yelp-sentiment-classifier-with-gpt2-large-embeds-energy-training/lewis-compr/mlm-beamsearch-v0-word-nps5-k10-beam5-allsat_primary-k5ojm8zg-negative-to-positive-grad_norm-k5ojm8zg/results_epsilon0.75.txt.cls',
               # 'outputs/sentiment/mlm-reranking/roberta-base-yelp-sentiment-classifier-with-gpt2-large-embeds-energy-training/lewis-compr/mlm-beamsearch-v0-word-nps5-k10-beam5-allsat_primary-z9nof4p7-negative-to-positive/results_epsilon0.75.txt.cls'
              ]

In [279]:
exp_names_0 = ['clsf_grad_norm','clsf_attn'
               #,'em_grad_norm','em_attn'
              ]

In [280]:
result_list_0=[]
for i, path in enumerate(result_paths_0):
    result=pd.read_json(path,lines=True)
    result=result.explode('generations')
    colname=exp_names_0[i]
    result[colname]=result['generations'].apply(lambda x: x['text'].rstrip('\n'))

    cls=pd.read_csv(cls_paths_0[i])
    result[f'{colname}_cls']=cls['sentiment_predictions']
    
    result=result[[colname,f'{colname}_cls']].copy()
    result_list_0.append(result)

In [332]:
result_0=pd.concat([ref_0, lewis_0]+result_list_0,axis=1)

In [333]:
result_0[[col for col in result_0.columns if '_cls' in col]].mean()

ref_cls               0.788
lewis_cls             0.832
clsf_grad_norm_cls    0.656
clsf_attn_cls         0.374
dtype: float64

In [334]:
## analyze diff between src and ref 

def get_token_diff(x, colname1, colname2):
    
    a_list = tokenizer.tokenize(x[colname1])
    b_list = tokenizer.tokenize(x[colname2])
    
    s = SequenceMatcher(None, a_list, b_list)
    
    replaced_deleted_tokens_ref = []
    for tag, i1, i2, j1, j2 in s.get_opcodes():
        if (tag == "replace") or (tag == "delete"):
            replaced_deleted_tokens_ref.extend(list(range(i1,i2)))
    return replaced_deleted_tokens_ref

In [335]:
result_0['edited_toks_ref']=result_0.apply(lambda x: get_token_diff(x,'src','ref'),axis=1)

In [336]:
result_0['edited_toks_lewis']=result_0.apply(lambda x: get_token_diff(x,'src','lewis'),axis=1)

In [337]:
result_0['edited_toks_clsf_grad_norm']=result_0.apply(lambda x: get_token_diff(x,'src','clsf_grad_norm'),axis=1)

In [338]:
result_0['edited_toks_clsf_attn']=result_0.apply(lambda x: get_token_diff(x,'src','clsf_attn'),axis=1)

In [339]:
import numpy as np

In [340]:
def locate_precision(x, colname1, colname2):
    if len(x[colname2])==0:
        return np.nan
    else:
        return len(set(x[colname1]).intersection(set(x[colname2])))/len(x[colname2])

result_0['locate_precision_lewis']=result_0.apply(lambda x: locate_precision(x, 'edited_toks_ref','edited_toks_lewis'),axis=1)
result_0['locate_precision_clsf_grad_norm']=result_0.apply(lambda x: locate_precision(x, 'edited_toks_ref','edited_toks_clsf_grad_norm'),axis=1)
result_0['locate_precision_clsf_attn']=result_0.apply(lambda x: locate_precision(x, 'edited_toks_ref','edited_toks_clsf_attn'),axis=1)

In [341]:
def locate_recall(x, colname1, colname2):
    if len(x[colname1])==0:
        return np.nan
    else:
        return len(set(x[colname1]).intersection(set(x[colname2])))/len(x[colname1])

result_0['locate_recall_lewis']=result_0.apply(lambda x: locate_recall(x, 'edited_toks_ref','edited_toks_lewis'),axis=1)
result_0['locate_recall_clsf_grad_norm']=result_0.apply(lambda x: locate_recall(x, 'edited_toks_ref','edited_toks_clsf_grad_norm'),axis=1)
result_0['locate_recall_clsf_attn']=result_0.apply(lambda x: locate_recall(x, 'edited_toks_ref','edited_toks_clsf_attn'),axis=1)

In [342]:
result_0['len_edited_tokens_ref']=result_0['edited_toks_ref'].apply(len)

In [343]:
result_0['len_edited_tokens_lewis']=result_0['edited_toks_lewis'].apply(len)

In [344]:
result_0['len_edited_tokens_clsf_attn']=result_0['edited_toks_clsf_attn'].apply(len)

In [345]:
result_0['len_edited_tokens_clsf_grad_norm']=result_0['edited_toks_clsf_grad_norm'].apply(len)

In [346]:
result_0[['len_edited_tokens_ref', 'len_edited_tokens_lewis', 'len_edited_tokens_clsf_attn', 'len_edited_tokens_clsf_grad_norm']].median()

len_edited_tokens_ref               6.0
len_edited_tokens_lewis             2.0
len_edited_tokens_clsf_attn         5.0
len_edited_tokens_clsf_grad_norm    6.0
dtype: float64

In [347]:
result_0[['locate_precision_lewis',
         'locate_precision_clsf_grad_norm',
         'locate_precision_clsf_attn']].mean()

locate_precision_lewis             0.784797
locate_precision_clsf_grad_norm    0.664760
locate_precision_clsf_attn         0.644929
dtype: float64

In [348]:
result_0[['locate_recall_lewis',
         'locate_recall_clsf_grad_norm',
         'locate_recall_clsf_attn']].mean()

locate_recall_lewis             0.350635
locate_recall_clsf_grad_norm    0.697967
locate_recall_clsf_attn         0.519730
dtype: float64

In [350]:
result_0[[col for col in result_0.columns if col.endswith('_cls')]].mean()

ref_cls               0.788
lewis_cls             0.832
clsf_grad_norm_cls    0.656
clsf_attn_cls         0.374
dtype: float64

In [351]:
result_0.loc[result_0['locate_precision_clsf_grad_norm'] >=0.5, 'clsf_grad_norm_cls'].mean()

0.6701030927835051

In [365]:
result_0.loc[result_0['locate_precision_clsf_grad_norm'] ==0., 'clsf_grad_norm_cls'].mean()

0.47058823529411764

In [352]:
result_0.loc[result_0['locate_recall_clsf_grad_norm'] >=0.5, 'clsf_grad_norm_cls'].mean()

0.7050359712230215

In [363]:
result_0.loc[result_0['locate_recall_clsf_grad_norm'] ==0., 'clsf_grad_norm_cls'].mean()

0.3

In [353]:
result_0.loc[result_0['locate_precision_clsf_attn'] >=0.5, 'clsf_attn_cls'].mean()

0.39572192513368987

In [366]:
result_0.loc[result_0['locate_precision_clsf_attn'] ==0., 'clsf_attn_cls'].mean()

0.2926829268292683

In [354]:
result_0.loc[result_0['locate_recall_clsf_attn'] >=0.5, 'clsf_attn_cls'].mean()

0.46440677966101696

In [361]:
result_0.loc[result_0['locate_recall_clsf_attn'] ==0., 'clsf_attn_cls'].mean()

0.2647058823529412

In [355]:
result_0.loc[result_0['locate_precision_lewis'] >=0.5, 'lewis_cls'].mean()

0.850356294536817

In [358]:
result_0.loc[result_0['locate_precision_lewis'] <0.5, 'lewis_cls'].mean()

0.7746478873239436

In [359]:
result_0.loc[result_0['locate_precision_lewis'] ==0, 'lewis_cls'].mean()

0.7021276595744681

In [356]:
result_0.loc[result_0['locate_recall_lewis'] >=0.5, 'lewis_cls'].mean()

0.9111111111111111

In [357]:
result_0.loc[result_0['locate_recall_lewis'] < 0.5, 'lewis_cls'].mean()

0.8044692737430168

In [360]:
result_0.loc[result_0['locate_recall_lewis'] == 0., 'lewis_cls'].mean()

0.6458333333333334

In [374]:
result_0.loc[(result_0['locate_recall_lewis']==0.) & 
            (result_0['lewis_cls']==1.),['src', 'ref','lewis']].values

array([['so basically tasted watered down .',
        "It didn't taste watered down at all. ",
        'so basically tasted watered and fresh .'],
       ['definitely disappointed that i could not use my birthday gift !',
        'definitely not disappointed that i could use my birthday gift !',
        'definitely recommend this place so much that i could not use my birthday gift !'],
       ['my pad thai tasted like thai rice noodles with barbeque sauce .',
        'the pad thai tasted like the best I have had in a while',
        'my pad thai and thai rice noodles with barbeque sauce were amazing .'],
       ['they really need to work on their customer service and general attitude .',
        "they don't need to work on their customer service and general attitude .",
        'they really seem to work on their customer service and general attitude .'],
       ['when i first came to phx ... yes this sounded indian to me .',
        'when i first came to phx ... yes this sounded americ

In [376]:
result_0.loc[(result_0['locate_recall_clsf_grad_norm']==0.) & 
            (result_0['clsf_grad_norm_cls']==1.),['src', 'ref','clsf_grad_norm']].values

array([['when i first came to phx ... yes this sounded indian to me .',
        'when i first came to phx ... yes this sounded american to me .',
        'so i was able to get this, and i am indian, and i'],
       ["we 'll see if that happens , but i 'm not holding my breath .",
        "we 'll see if that doesn't happen , but i 'm not holding my breath that it won't .",
        "I 'll see if that happens, but i'm still on my own."],
       ['no wonder these restaurants are closing up all around the valley .',
        "no wonder these restaurants aren't closing up all around the valley .",
        'The new food restaurants are popping up all around the country.']],
      dtype=object)

In [377]:
result_0.loc[(result_0['locate_recall_clsf_grad_norm']==0.) & 
            (result_0['clsf_grad_norm_cls']==0.),['src', 'ref','clsf_grad_norm']].values

array([['i know i should have sent this back and walk out .',
        "i know i shouldn't have sent this back and walked out .",
        'I think I should have sent this back and walk away.'],
       ['so who do i call to get my money back .',
        'so who do i call to give my money to .',
        'So who do i call to get my money back?'],
       ['the sandwich was not that great .', 'the sandwich was great .',
        'The sandwich was not that good.'],
       ['the only thing i was offered was a free dessert .',
        "the only thing i wasn't offered was a free dessert .",
        'The best thing I was offered was a free lunch box'],
       ['management does not seem to care .',
        'management really seem to care .', 'He does not seem to care.'],
       ['service was average but could not make up for the poor food and drink .',
        'service was above average as well as the food and drink .',
        'It is good but could not make up for the poor food and drink.'],
     

In [378]:
result_0.loc[(result_0['locate_recall_clsf_grad_norm']>0.5) & 
            (result_0['clsf_grad_norm_cls']==0.),['src', 'ref','clsf_grad_norm']].values

array([["i ca n't believe how inconsiderate this pharmacy is .",
        'This pharmacy is really considerate.',
        "I don't know what it's like in this pharmacy, but"],
       ['just left and took it off the bill .',
        'just left and put it on the bill .',
        'Download this and get it from the store.'],
       ['but it probably sucks too !',
        "but it probably doesn't suck too !", "but it probably won't."],
       ["said we could n't sit at the table if we were n't ordering dinner .",
        'they said we could sit at the table with no hesitation',
        "Do you really want to sit at the table when we weren't ordering food?"],
       ['the burgers were over cooked to the point the meat was crunchy .',
        'the burgers were cooked perfectly and the meat was juicy',
        'The burgers have been cooked to the point the meat is almost cooked.'],
       ['so i asked for the card to be refunded .',
        'I might consider asking for the card to be refunded. 

In [36]:
ref_1 = pd.read_csv('data/Sentiment-and-Style-Transfer/data/yelp/reference.1',sep='\t',header=None,names=['src','ref'])

In [37]:
with open('lewis/model_outputs/yelp/neg.out', 'r') as f:
    lewis_1 = pd.DataFrame({"lewis":[line.rstrip('\n') for line in f.readlines()]})

In [22]:
result_paths_1 = ['outputs/sentiment/mlm-reranking/roberta-base-yelp-lewis-sentiment-classifier-with-gpt2-large-embeds-binary/lewis-compr/mlm-beamsearch-v0-word-nps5-k10-beam5-allsat_primary-ij4hre36-positive-to-negative-grad_norm-ij4hre36/outputs_epsilon0.75.txt',
                'outputs/sentiment/mlm-reranking/roberta-base-yelp-lewis-sentiment-classifier-with-gpt2-large-embeds-binary/lewis-compr/mlm-beamsearch-v0-word-nps5-k10-beam5-allsat_primary-qdgh9mv2-positive-to-negative/outputs_epsilon0.75.txt',
               'outputs/sentiment/mlm-reranking/roberta-base-yelp-sentiment-classifier-with-gpt2-large-embeds-energy-training/lewis-compr/mlm-beamsearch-v0-word-nps5-k10-beam5-allsat_primary-hw52a3y0-positive-to-negative-grad_norm-hw52a3y0/outputs_epsilon0.75.txt',
               'outputs/sentiment/mlm-reranking/roberta-base-yelp-sentiment-classifier-with-gpt2-large-embeds-energy-training/lewis-compr/mlm-beamsearch-v0-word-nps5-k10-beam5-allsat_primary-bff2akpu-positive-to-negative/outputs_epsilon0.75.txt']

In [23]:
result_list_1=[]
for path in result_paths_1:
    result=pd.read_json(path,lines=True)
    result=result.explode('generations')
    colname='/'.join([path.split('/')[3],path.split('/')[5]])
    result[colname]=result['generations'].apply(lambda x: x['text'].rstrip('\n'))
    result=result[[colname]].copy()
    result_list_1.append(result)

In [38]:
result_1=pd.concat([ref_1,lewis_1]+result_list_1,axis=1)

In [76]:
from Levenshtein import editops, opcodes

In [79]:
from transformers import AutoTokenizer

In [80]:
tokenizer = AutoTokenizer.from_pretrained("/shared/s3/lab07/hyeryung/loc_edit/models/roberta-base-yelp-lewis-sentiment-classifier-with-gpt2-large-embeds-binary/step_280_best_checkpoint/")

In [88]:
result_0['src']=result_0['src'].apply(lambda x: tokenizer.decode(tokenizer.encode(x,add_special_tokens=False)))

In [89]:
result_0['ref']=result_0['ref'].apply(lambda x: tokenizer.decode(tokenizer.encode(x,add_special_tokens=False)))

In [90]:
result_0['lewis']=result_0['lewis'].apply(lambda x: tokenizer.decode(tokenizer.encode(x,add_special_tokens=False)))

In [99]:
from difflib import context_diff, Differ, SequenceMatcher

In [133]:
import random

In [235]:
sample_id = random.randint(0,len(result_0)-1)

In [236]:
a_list = tokenizer.tokenize(result_0.src[sample_id])
b_list = tokenizer.tokenize(result_0.iloc[sample_id, 1])

print(f"src: {result_0.src[sample_id]}")
print(f"ref: {result_0.iloc[sample_id, 1]}")

s = SequenceMatcher(None, a_list, b_list)

replaced_deleted_tokens_ref = []
for tag, i1, i2, j1, j2 in s.get_opcodes():
    if tag != "equal":
        print('{:7}   a[{}:{}] --> b[{}:{}] {!r:>8} --> {!r}'.format(
            tag, i1, i2, j1, j2, a_list[i1:i2], b_list[j1:j2]))
    if (tag == "replace") or (tag == "delete"):
        replaced_deleted_tokens_ref.extend(list(range(i1,i2)))

src: ate kicked ass, marquee, not so much.
ref: ate kicked ass, marquee, pretty good too.
replace   a[6:9] --> b[6:9] ['Ġnot', 'Ġso', 'Ġmuch'] --> ['Ġpretty', 'Ġgood', 'Ġtoo']


In [237]:
replaced_deleted_tokens_ref

[6, 7, 8]

In [238]:
a_list = tokenizer.tokenize(result_0.src[sample_id])
b_list = tokenizer.tokenize(result_0.iloc[sample_id, 2])

print(f"src: {result_0.src[sample_id]}")
print(f"lewis: {result_0.iloc[sample_id, 2]}")

s = SequenceMatcher(None, a_list, b_list)

replaced_deleted_tokens_lewis = []
for tag, i1, i2, j1, j2 in s.get_opcodes():
    if tag != "equal":
        print('{:7}   a[{}:{}] --> b[{}:{}] {!r:>8} --> {!r}'.format(
            tag, i1, i2, j1, j2, a_list[i1:i2], b_list[j1:j2]))
    if (tag == "replace") or (tag == "delete"):
        replaced_deleted_tokens_lewis.extend(list(range(i1,i2)))

src: ate kicked ass, marquee, not so much.
lewis: ate kicked ass, marquee, and so did everyone.
replace   a[6:7] --> b[6:7] ['Ġnot'] --> ['Ġand']
replace   a[8:9] --> b[8:10] ['Ġmuch'] --> ['Ġdid', 'Ġeveryone']


In [239]:
print(f"num overlapping replaced or deleted tokens: {len(set(replaced_deleted_tokens_ref).intersection(replaced_deleted_tokens_lewis))}")

num overlapping replaced or deleted tokens: 2


In [240]:
a_list = tokenizer.tokenize(result_0.src[sample_id])
b_list = tokenizer.tokenize(result_0.iloc[sample_id, 3])

print(f"src: {result_0.src[sample_id]}")
print(f"{result_0.columns[3]}: {result_0.iloc[sample_id, 3]}")

s = SequenceMatcher(None, a_list, b_list)

replaced_deleted_tokens_3 = []
for tag, i1, i2, j1, j2 in s.get_opcodes():
    if tag != "equal":
        print('{:7}   a[{}:{}] --> b[{}:{}] {!r:>8} --> {!r}'.format(
            tag, i1, i2, j1, j2, a_list[i1:i2], b_list[j1:j2]))
    if (tag == "replace") or (tag == "delete"):
        replaced_deleted_tokens_3.extend(list(range(i1,i2)))

src: ate kicked ass, marquee, not so much.
roberta-base-yelp-lewis-sentiment-classifier-with-gpt2-large-embeds-binary/mlm-beamsearch-v0-word-nps5-k10-beam5-allsat_primary-866hmm6j-negative-to-positive-grad_norm-866hmm6j: I, on the and, and so on.
replace   a[0:3] --> b[0:1] ['ate', 'Ġkicked', 'Ġass'] --> ['I']
replace   a[4:5] --> b[2:5] ['Ġmarquee'] --> ['Ġon', 'Ġthe', 'Ġand']
replace   a[6:7] --> b[6:7] ['Ġnot'] --> ['Ġand']
replace   a[8:9] --> b[8:9] ['Ġmuch'] --> ['Ġon']


In [241]:
print(f"num overlapping replaced or deleted tokens: {len(set(replaced_deleted_tokens_ref).intersection(replaced_deleted_tokens_3))}")

num overlapping replaced or deleted tokens: 2


In [242]:
a_list = tokenizer.tokenize(result_0.src[sample_id])
b_list = tokenizer.tokenize(result_0.iloc[sample_id, 4])

print(f"src: {result_0.src[sample_id]}")
print(f"{result_0.columns[4]}: {result_0.iloc[sample_id, 4]}")

s = SequenceMatcher(None, a_list, b_list)

replaced_deleted_tokens_4 = []
for tag, i1, i2, j1, j2 in s.get_opcodes():
    if tag != "equal":
        print('{:7}   a[{}:{}] --> b[{}:{}] {!r:>8} --> {!r}'.format(
            tag, i1, i2, j1, j2, a_list[i1:i2], b_list[j1:j2]))
    if (tag == "replace") or (tag == "delete"):
        replaced_deleted_tokens_4.extend(list(range(i1,i2)))

src: ate kicked ass, marquee, not so much.
roberta-base-yelp-lewis-sentiment-classifier-with-gpt2-large-embeds-binary/mlm-beamsearch-v0-word-nps5-k10-beam5-allsat_primary-9fjc2pfg-negative-to-positive: It's not, but, not so much.
replace   a[0:3] --> b[0:3] ['ate', 'Ġkicked', 'Ġass'] --> ['It', "'s", 'Ġnot']
replace   a[4:5] --> b[4:5] ['Ġmarquee'] --> ['Ġbut']


In [243]:
print(f"num overlapping replaced or deleted tokens: {len(set(replaced_deleted_tokens_ref).intersection(replaced_deleted_tokens_4))}")

num overlapping replaced or deleted tokens: 0


In [244]:
a_list = tokenizer.tokenize(result_0.src[sample_id])
b_list = tokenizer.tokenize(result_0.iloc[sample_id, 5])

print(f"src: {result_0.src[sample_id]}")
print(f"{result_0.columns[5]}: {result_0.iloc[sample_id, 5]}")

s = SequenceMatcher(None, a_list, b_list)

replaced_deleted_tokens_5 = []
for tag, i1, i2, j1, j2 in s.get_opcodes():
    if tag != "equal":
        print('{:7}   a[{}:{}] --> b[{}:{}] {!r:>8} --> {!r}'.format(
            tag, i1, i2, j1, j2, a_list[i1:i2], b_list[j1:j2]))
    if (tag == "replace") or (tag == "delete"):
        replaced_deleted_tokens_5.extend(list(range(i1,i2)))

src: ate kicked ass, marquee, not so much.
roberta-base-yelp-sentiment-classifier-with-gpt2-large-embeds-energy-training/mlm-beamsearch-v0-word-nps5-k10-beam5-allsat_primary-k5ojm8zg-negative-to-positive-grad_norm-k5ojm8zg: In fact, the situation is not so bad.
replace   a[0:3] --> b[0:2] ['ate', 'Ġkicked', 'Ġass'] --> ['In', 'Ġfact']
replace   a[4:6] --> b[3:6] ['Ġmarquee', ','] --> ['Ġthe', 'Ġsituation', 'Ġis']
replace   a[8:9] --> b[8:9] ['Ġmuch'] --> ['Ġbad']


In [245]:
print(f"num overlapping replaced or deleted tokens: {len(set(replaced_deleted_tokens_ref).intersection(replaced_deleted_tokens_5))}")

num overlapping replaced or deleted tokens: 1


In [246]:
a_list = tokenizer.tokenize(result_0.src[sample_id])
b_list = tokenizer.tokenize(result_0.iloc[sample_id, 6])

print(f"src: {result_0.src[sample_id]}")
print(f"{result_0.columns[6]}: {result_0.iloc[sample_id, 6]}")

s = SequenceMatcher(None, a_list, b_list)

replaced_deleted_tokens_6 = []
for tag, i1, i2, j1, j2 in s.get_opcodes():
    if tag != "equal":
        print('{:7}   a[{}:{}] --> b[{}:{}] {!r:>8} --> {!r}'.format(
            tag, i1, i2, j1, j2, a_list[i1:i2], b_list[j1:j2]))
    if (tag == "replace") or (tag == "delete"):
        replaced_deleted_tokens_6.extend(list(range(i1,i2)))

src: ate kicked ass, marquee, not so much.
roberta-base-yelp-sentiment-classifier-with-gpt2-large-embeds-energy-training/mlm-beamsearch-v0-word-nps5-k10-beam5-allsat_primary-z9nof4p7-negative-to-positive: And the ass, too, and so on.
replace   a[0:2] --> b[0:2] ['ate', 'Ġkicked'] --> ['And', 'Ġthe']
replace   a[4:5] --> b[4:5] ['Ġmarquee'] --> ['Ġtoo']
replace   a[6:7] --> b[6:7] ['Ġnot'] --> ['Ġand']
replace   a[8:9] --> b[8:9] ['Ġmuch'] --> ['Ġon']


In [247]:
print(f"num overlapping replaced or deleted tokens: {len(set(replaced_deleted_tokens_ref).intersection(replaced_deleted_tokens_6))}")

num overlapping replaced or deleted tokens: 2
